# 要約 
このJupyter Notebookは、Kaggleの「20の質問」ゲームに参加するためのAIエージェントを構築することを目的としています。このノートブックでは、特に大規模言語モデル（LLM）を扱うために、モジュールのインストールやモデルのダウンロード、エージェントのロジックを実装する方法に焦点を当てています。

### 問題
ノートブックは、「20の質問」ゲームで使用される質問者と回答者のAIエージェントを設計し、特にジオメトリックな推論が求められる環境での効率的な質問生成と推測を支援することを目指しています。最終的には、これらのエージェントが対戦を通じて他のチームに勝利することが求められています。

### 解決手法
このノートブックでは以下のような手法を用いて問題を解決しています：

1. **ライブラリのインポートとモデルの取得**:
    - `transformers`および`bitsandbytes`ライブラリをインストールし、大規模言語モデルを使用するための環境を構築しています。
    - Hugging Face Hubから、Gemma 2という特定のモデルをダウンロードし、ローカル環境で使用できるようにデバイスマッピングと量子化設定を行っています。

2. **モデルの設定**:
    - モデルの各層をGPUに分散させるためのデバイスマッピングを定義し、メモリ効率を向上させるための設定を行います。
    - 使用するトークナイザーをモデルからロードし、生成された出力に対して後処理を行う関数が定義されています。

3. **エージェントのロジックの実装**:
    - ゲームのロジックを処理する`Robot`クラスが定義され、質問生成 (`asker`)、推測 (`guessing`)、および回答 (`answering`) のためのメソッドが実装されています。
    - 各メソッドは、AIエージェントが与えられたメッセージ履歴に基づいて適切な出力を生成します。

4. **ゲームシミュレーション**:
    - 独自の`Observation`クラスを使用して、ゲームの進行状況を追跡し、質問、回答、推測の履歴を管理します。
    - 20ラウンドの間、エージェントが与えられたキーワードに対して質問や推測を行い、どのようにエージェントがゲームを進めるかを示しています。

5. **提出の試み**:
    - 最後に、出力ディレクトリの容量に関する問題で提出ができなかったことが言及されています。

### 使用されたライブラリ
- **Transformers**: 自然言語処理のための強力なライブラリで、大規模言語モデルのロードやトークナイザーの生成に使用。
- **BitsAndBytes**: モデルの量子化に利用され、計算資源の効率的な使用を促進します。

このノートブックは、AIエージェントが「20の質問」ゲームでどのように機能するか、その背景にある技術や手法を深く探るものとなっています。

---


# 用語概説 
以下は、与えられたJupyter Notebookの内容に関連した、機械学習・深層学習の初心者がつまずきそうな専門用語の解説です。

### 専門用語の解説

1. **bitsandbytes**:
   - 主にメモリ効率の良いディープラーニングモデルを扱うためのライブラリ。主に、量子化やモデルの圧縮を行い、GPUメモリの使用量を削減するために利用される。

2. **Hugging Face**:
   - 自然言語処理に特化した機械学習モデルやデータセットが集約されたプラットフォーム。特に、トランスフォーマーアーキテクチャに基づくモデルのリポジトリが豊富で、モデルの簡単な利用が可能。

3. **device_map**:
   - モデルの異なる層を異なるデバイス（CPUやGPUなど）に割り当てる際に使用される設定。メモリと計算資源を最適に利用するために役立つ。

4. **torch.backends.cuda.enable_mem_efficient_sdp**:
   - PyTorchにおいて、メモリ効率の良いShared Data Parallel（SDP）を有効または無効にするための関数。この設定によって、異なるデバイス上でのメモリ使用の効率を調整できる。

5. **quantization**:
   - モデルのパラメータを浮動小数点から整数に変換することで、モデルサイズを縮小し、推論速度を向上させる技術。通常は、計算資源の制約がある環境での運用を考慮して使用される。

6. **gradient_checkpointing**:
   - 大規模なディープラーニングモデルのトレーニング時に、メモリ使用量を削減する手法。バックプロパゲーションの過程で全ての中間勾配を保存せず、必要な時に再計算することで、メモリの負担を減らす。

7. **tokenization**:
   - テキストをモデルが理解できる形式に変換するプロセス。自然言語処理において、単語やフレーズを「トークン」と呼ばれる単位に分割し、それを数値にマッピングする。

8. **<eos>トークン**:
   - "End of Sentence"の略称で、文章の終了を示す特別なトークン。モデルが生成したテキストの中でこのトークンが現れると、生成プロセスが終了する。

9. **Kaggle Dreams**:
   - Kaggle上でのシミュレーションやプレイ環境を提供するフレームワーク。ただし、特定の言語や専門的な環境設定が必要な場合があるので、事前の理解が求められます。

10. **エージェント**:
    - タスクを遂行する AI システムやプログラム。競技環境やゲームにおいて、プレーヤーとして振る舞うものを指す。

11. **モノレポ**:
    - 複数のプロジェクトを一つのリポジトリ（レポジトリ）内で管理するアプローチ。複雑なプロジェクトのバージョン管理や依存関係の管理を容易にします。

以上の用語は、特に事前に知識がないと悪戦苦闘する可能性がある分野のものです。理論や基本を理解していても、実務での経験がないと見逃しがちなので、十分な理解を深めることをお勧めします。

---


In [ ]:
%%bash
mkdir -p /kaggle/working/submission
pip install bitsandbytes accelerate
pip install -U transformers

**秘密を保持する:** アドオン>シークレット


In [ ]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None

try:
    HF_TOKEN = secrets.get_secret("hf_token")
except:
    pass

**LLMをローカルにダウンロードする。** gemma2は入力から呼び出して実行できなかったので、ローカルにダウンロードしました。


In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/working/submission/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)  # 既存のモデルパスがあれば削除
g_model_path.mkdir(parents=True)  # 新規ディレクトリを作成

# Hugging Face Hubからモデルのスナップショットをダウンロード
snapshot_download(
    repo_id="google/gemma-2-9b-it",
    ignore_patterns="original*",
    local_dir=g_model_path,
    local_dir_use_symlinks=False,
    token=globals().get("HF_TOKEN", None)  # グローバルに定義されたトークンを取得
)

In [ ]:
!ls -l /kaggle/working/submission/model  # モデルディレクトリの内容をリスト表示

**モデル層のデバイスマッピング**
大規模モデルを扱う際に、複数のデバイスに層を分散させることで、メモリと計算リソースをより効率的に管理するのに役立ちます。device_maps変数は、モデルのどの層がどのデバイスに割り当てられているかを指定するために使用されます。

torch.backends.cuda.enable_mem_efficient_sdp(False)を使用すると、PyTorchのメモリ効率の良いSDPを無効にすることができ、特定のモデルやデバッグの目的で必要になる場合があります。この設定により、異なるハードウェアセットアップ間でのパフォーマンスや一貫性を向上させることができます。

[Gemma 2のプロンプト方法](https://huggingface.co/blog/gemma2#how-to-prompt-gemma-2)


In [ ]:
%%writefile -a submission/main.py

import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

torch.backends.cuda.enable_mem_efficient_sdp(False)  # メモリ効率の良いSDPを無効にする

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "model")  # Kaggleのエージェントパスにモデルがあるか確認
else:
    model_id = "/kaggle/working/submission/model"  # ローカルモデルパス

# モデル層のデバイスマッピング
device_maps = [('model.layers.0', 0),
 ('model.layers.1', 0),
 ('model.layers.2', 0),
 ('model.layers.3', 0),
 ('model.layers.4', 0),
 ('model.layers.5', 0),
 ('model.layers.6', 0),
 ('model.layers.7', 0),
 ('model.layers.8', 0),
 ('model.layers.9', 0),
 ('model.layers.10', 0),
 ('model.layers.11', 0),
 ('model.layers.12', 0),
 ('model.layers.13', 0),
 ('model.layers.14', 0),
 ('model.layers.15', 0),
 ('model.layers.16', 0),
 ('model.layers.17', 0),
 ('model.layers.18', 0),
 ('model.layers.19', 1),
 ('model.layers.20', 1),
 ('model.layers.21', 1),
 ('model.layers.22', 1),
 ('model.layers.23', 1),
 ('model.layers.24', 1),
 ('model.layers.25', 1),
 ('model.layers.26', 1),
 ('model.layers.27', 1),
 ('model.layers.28', 1),
 ('model.layers.29', 1),
 ('model.layers.30', 1),
 ('model.layers.31', 1),
 ('model.layers.32', 1),
 ('model.layers.33', 1),
 ('model.layers.34', 1),
 ('model.layers.35', 1),
 ('model.layers.36', 1),
 ('model.layers.37', 1),
 ('model.layers.38', 1),
 ('model.layers.39', 1),
 ('model.layers.40', 1),
 ('model.layers.41', 1),
 ('model.embed_tokens', 1),
 ('model.layers', 1)]

# 量子化設定
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)  # トークナイザーをモデルIDから取得
id_eot = tokenizer.convert_tokens_to_ids(["<eos>"])[0]  # <eos>トークンのIDを取得

# 各層のデバイスを設定
device = {layer:gpu_mem for (layer,gpu_mem) in device_maps}
config = AutoConfig.from_pretrained(model_id)  # モデル設定を取得
config.gradient_checkpointing = True  # 勾配チェックポイントを有効にする
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", quantization_config=quantization_config,
                                             device_map="auto", trust_remote_code=True, config=config)

# 回答を生成する関数
def generate_answer(template):
    input_ids = tokenizer(template, return_tensors="pt").to("cuda")  # テンプレートをトークン化してGPUに転送
    output_ids = model.generate(**input_ids, max_new_tokens=15).squeeze()  # モデルから出力を生成
    start_gen = input_ids.input_ids.shape[1]  # 入力の長さを取得
    output_ids = output_ids[start_gen:]  # 出力を切り出す
    if id_eot in output_ids:
        stop = output_ids.tolist().index(id_eot)  # <eos>の位置を取得
        output = tokenizer.decode(output_ids[:stop])  # またトークンをデコード
    else:
        output = tokenizer.decode(output_ids)  # トークンをデコード
    output = re.sub('\n', '', output)  # 改行を削除
    output = re.sub(' <end_of_turn>', '', output)  # 不要なトークンを削除
    output = re.sub('<end_of_turn>', '', output)  # 不要なトークンを削除
    return output

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            output = self.asker(obs)  # 質問者のロジックを呼び出す
        if mode == "answering":
            output = self.answerer(obs)  # 回答者のロジックを呼び出す
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"  # 否定形を整形
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"  # 否定形を整形
            else:
                output = "yes"  # デフォルトでは「yes」にする
        if mode == "guessing":
            output = self.asker(obs)  # 推測者のロジックを呼び出す
        return output

    def asker(self, obs):
        sys_prompt = """
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。
        このゲームでは、回答者がキーワードを考え、質問者がそれに対してはい・いいえの質問をします。
        キーワードは特定の「もの」や「場所」です。
        """
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """
            20の質問をプレイします。あなたは質問者の役割を果たします。
            はい・いいえで答えられる質問をしてください。
            キーワードがモロッコであると仮定した場合の例をご覧ください：
            例：
            <あなた: それは国ですか？
            ユーザー: はい
            あなた: それはヨーロッパにありますか？
            ユーザー: いいえ
            あなた: それはアフリカにありますか？
            ユーザー: はい
            あなた: そこに住むほとんどの人は肌の色が暗いですか？
            ユーザー: いいえ
            あなた: それは「m」で始まる国名ですか？
            ユーザー: はい
            あなた: モロッコですか？
            ユーザー: はい。>
            ユーザーが単語を選びました。最初の質問をしてください！
            短く、冗長にならないように、一つの質問だけをして下さい！
            """
        
            chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

            if len(obs.questions)>=1:
                    for q, a in zip(obs.questions, obs.answers):
                        chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                        chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
                    
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""質問: {q}\n回答: {a}\n"""  # 質問と回答の履歴を記録
            guess_prompt =  sys_prompt + f"""
            現在のゲームの状況は以下の通りです:\n{conv}
            この会話に基づいて、単語を推測してください。また、推測には1つの単語だけを答えて、冗長にならないように。
            """
            chat_template = f"""<start_of_turn>system\n{guess_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

        output = generate_answer(chat_template)        
        return output
    
    def answerer(self, obs):
        ask_prompt = f"""
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者がそれに対してはい・いいえの質問をします。
        キーワードは特定の場所や物です。
        今、ユーザーが推測すべき単語は"{obs.keyword}"で、カテゴリーは"{obs.category}"です。
        ユーザーがキーワードを推測する際には、以下の手順を参考にしてください。
        例：
        <ユーザー: それは場所ですか？
        あなた: はい
        ユーザー: それはヨーロッパにありますか？
        あなた: いいえ
        ユーザー: それはアフリカにありますか？
        あなた: はい
        ユーザー: そこに住むほとんどの人は肌の色が暗いですか？
        あなた: いいえ
        ユーザー: それは「m」で始まる国名ですか？
        あなた: はい
        ユーザー: モロッコですか？
        あなた: はい。>
        """
        chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>user\n"
        chat_template += f"{obs.questions[0]}"
        chat_template += "<start_of_turn>model\n"
        if len(obs.answers)>=1:
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        output = generate_answer(chat_template)
        return output

robot = Robot()

# エージェントのメイン関数
def agent(obs, cfg):
    
    if obs.turnType =="ask":
        response = robot.on(mode = "asking", obs = obs)  # 質問の処理
        
    elif obs.turnType =="guess":
        response = robot.on(mode = "guessing", obs = obs)  # 推測の処理
        
    elif obs.turnType =="answer":
        response = robot.on(mode = "answering", obs = obs)  # 回答の処理
        
    if response == None or len(response)<=1:
        response = "yes"  # デフォルトの応答を設定
        
    return response

In [ ]:
# def simple_agent1(obs, cfg):
#     # エージェントが推測者で、turnTypeが「ask」の場合
#     if obs.turnType == "ask":
#         response_list = [
#             'それはアフリカにありますか？',
#             'それはアメリカにありますか？',
#             'それはアジアにありますか？',
#             'それはオセアニアにありますか？',
#             'それは東ヨーロッパにありますか？',
#             'それは北ヨーロッパにありますか？',
#             'それは南ヨーロッパにありますか？',
#             'それは西ヨーロッパにありますか？',
#             'それは日本ですか？'
#         ]
# #         response = response_list[len(obs.questions)]
#         response = random.choice(response_list)  # ランダムに質問を選択
#     elif obs.turnType == "guess":
#         response = "duck"  # デフォルトの推測
#     elif obs.turnType == "answer":
#         response = random.choices(["はい", "いいえ"])[0]  # ランダムに回答
#     return response

In [ ]:
# %%time

# import random
# from kaggle_environments import make
# agent = "/kaggle/working/submission/main.py"
# env = make("llm_20_questions", debug=True)
# game_output = env.run(agents=[agent, simple_agent1, simple_agent1, simple_agent1])

In [ ]:
# env.render(mode="ipython", width=600, height=500)  # 環境を描画

In [ ]:
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py  # キーワードリストをダウンロード

In [ ]:
import json
import pandas as pd
from submission.main import agent
from keywords_local import KEYWORDS_JSON

class Observation:
    def __init__(self):
        self.step = 0
        self.role = "guesser"  # デフォルトの役割は推測者
        self.turnType = "ask"  # デフォルトのターンタイプは質問
        self.keyword = "Japan"  # デフォルトのキーワード
        self.category = "country"  # デフォルトのカテゴリー
        self.questions = []  # 質問リスト
        self.answers = []  # 回答リスト
        self.guesses = []  # 推測リスト
        
# キーワードリストをデータフレームに変換する関数
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)  # JSONを読み込む

    keyword_list = []
    category_list = []
    alts_list = []

    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)  # キーワードをリストに追加
            category_list.append(json_data[i]['category'])  # カテゴリーをリストに追加
            alts_list.append(json_data[i]['words'][j]['alts'])  # 代替ワードをリストに追加

    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])  # データフレームを作成
    data_pd['keyword'] = keyword_list
    data_pd['category'] = category_list
    data_pd['alts'] = alts_list
    
    return data_pd
    
keywords_df = create_keyword_df(KEYWORDS_JSON)  # キーワードデータフレームを作成

In [ ]:
obs = Observation()
cfg = "_"

sample_df = keywords_df.sample()  # サンプルを取得
obs.keyword = sample_df["keyword"].values[0]  # サンプルのキーワードを設定
obs.category = sample_df["category"].values[0]  # サンプルのカテゴリーを設定
alts_list = sample_df["alts"].values[0]  # サンプルの代替ワードを設定
alts_list.append(obs.keyword)  # サンプルのキーワードを代替ワードリストに追加

print(f"キーワード:{obs.keyword}")  # キーワードを表示

for round in range(20):
    obs.step = round+1  # 現在のラウンドをインクリメント
    
    obs.role = "guesser"  # 役割を推測者に設定
    obs.turnType = "ask"  # ターンタイプを質問に設定
    question = agent(obs, cfg)  # 質問を生成
    obs.questions.append(question)  # 質問リストに追加
    
    obs.role = "answerer"  # 役割を回答者に設定
    obs.turnType = "answer"  # ターンタイプを回答に設定
    answer = agent(obs, cfg)  # 回答を生成
    obs.answers.append(answer)  # 回答リストに追加
    
    obs.role = "guesser"  # 役割を推測者に設定
    obs.turnType = "guess"  # ターンタイプを推測に設定
    guess = agent(obs, cfg)  # 推測を生成
    obs.guesses.append(guess)  # 推測リストに追加
    
    print(f"ラウンド: {round+1}")  # 現在のラウンドを表示
    print(f"質問: {question}")  # 質問を表示
    print(f"回答: {answer}")  # 回答を表示
    print(f"推測: {guess}")  # 推測を表示
    
    if guess in alts_list:  # 推測が代替ワードリストに含まれているか確認
        print("勝ちました！！")  # 勝利メッセージを表示
        break

**提出できませんでした...** 出力ディレクトリが容量を超えたため、tarファイルを作成できませんでした。


In [ ]:
# !apt install pigz pv > /dev/null

In [ ]:
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .

In [ ]:
# # tar.gzファイルの中身を確認するために

# import tarfile
# tar = tarfile.open("/kaggle/working/submission.tar.gz")
# for file in tar.getmembers():
#     print(file.name)  # tarファイル内の全てのファイルを表示

---

# コメント 

> ## Yuang Wu
> 
> どのようにして推測時に1つの単語を出力するのですか？…私はgemma 7b-it 3を使用しましたが、エージェントは推測時に2つ以上の単語を生成し、質問も奇妙になります。 
> 
> 
> 
> > ## Valentin Baltazar
> > 
> > 同様です…知りたいです。gemma2はそれほど優れているのでしょうか？
> > 
> > 
> > > ## KasaharaTopic 作者
> > > 
> > > gemma7b-itを使用しているとき、同じ問題が発生しました。少し改善されたようですが、私の実験ではllama3が正しい長さの単語を出力できる最高のモデルだと思います。
> > > 
> > > 
> > > > ## Yuang Wu
> > > > 
> > > > gemma 2 9bに変更したところ、状況が改善しました。
> > > > 
> > > > 

---

> ## ravi tanwar
> 
> このコンペティションに提出することに成功しましたか？
> 
> > ## KasaharaTopic 作者
> > 
> > submission.tarファイルを作成できなかったため、出力ディレクトリが容量を超えました。だから、提出できませんでした。
> > 
> > 提出できたとしても、このプロンプトには改善の余地があると思います。質問をうまく生成しませんから。
> > 
> > 